<a href="https://colab.research.google.com/github/AUT-Student/NLP-HW1/blob/main/NLP_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!gdown --id 16C0_9i0io43VfABV3-uukUjJYlM6k-2U
!unzip /content/HW1-datasets.zip

Downloading...
From: https://drive.google.com/uc?id=16C0_9i0io43VfABV3-uukUjJYlM6k-2U
To: /content/HW1-datasets.zip
100% 3.14M/3.14M [00:00<00:00, 195MB/s]
Archive:  /content/HW1-datasets.zip
   creating: content/HW1-datasets/
  inflating: content/HW1-datasets/train.txt  
  inflating: content/HW1-datasets/valid.txt  
  inflating: content/HW1-datasets/test_incomplete.txt  
  inflating: content/HW1-datasets/test.txt  
  inflating: content/HW1-datasets/test_incomplete_gold.txt  


In [12]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [4]:
file = open("/content/content/HW1-datasets/train.txt")

vocabulary = set()

for i, line in enumerate(file.readlines()):
  if i < 30*1000:
    words = line.split()

    for word in words:
      vocabulary.add(word)

In [5]:
len(vocabulary)

21188

In [6]:
sorted_vocabulary = sorted(vocabulary)

In [54]:
class BigramNLM(keras.Model):
  def __init__(self, vocabulary):
    super().__init__()
    self.vocabulary = vocabulary
    self.model = keras.models.Sequential([
                                          keras.layers.Input((2,), name="Input"),
                                          keras.layers.Embedding(len(self.vocabulary), 128, name="Embedding"),
                                          keras.layers.Flatten(name="Flatten"),
                                          keras.layers.Dense(units=256, name="Hidden"),
                                          keras.layers.Dense(units=len(self.vocabulary), activation="softmax", name="Output"),
    ])

  def convert_indexes_to_words(self, indexes):
    return np.array(list(map(lambda x:self.vocabulary[x], indexes)))

  def convert_word_to_index(self, word):
    return self.vocabulary.index(word)

  def call(self, inputs):
    model_outputs = self.model(inputs)
    predicted_word_indexes = np.argmax(model_outputs, axis=1)
    predicted_words = self.convert_indexes_to_words(predicted_word_indexes)  
    return predicted_words

In [55]:
bnlm = BigramNLM(vocabulary=sorted_vocabulary)

In [56]:
bnlm.call(tf.constant([
                        [4, 5],
                        [4, 7],
                        [6, 2]
                        ]))

array(['سازنده', 'متراکم', 'ریایی'], dtype='<U6')

In [34]:
bnlm.call(tf.constant([
                        [1, 2],
                        [3, 4],
                        [1, 2]
                        ]))

tf.Tensor(
[[1 2]
 [3 4]
 [1 2]], shape=(3, 2), dtype=int32)


array(['دنیاکوشم', 'مپای', 'دنیاکوشم'], dtype='<U8')

In [90]:
bnlm.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])